In [1]:
# FX Scoreboard — Notebook Signaux (version adaptée + bypass fichiers externes)
# ------------------------------------------------------------
# Fichiers de base attendus dans DATA_DIR :
#   FX.xlsx    : Dates, EURUSD, EURGBP, EURCHF, EURJPY
#   CPI.xlsx   : Dates, US_CPI, EU_CPI
#   GDP.xlsx   : Dates, US_GDP, EU_GDP
#   Cash.xlsx  : Dates, Cash
# Fichiers optionnels (bypass) :
#   Rates.xlsx       : Dates, USD_rate_1m, GBP_rate_1m, CHF_rate_1m, JPY_rate_1m
#   Macro_extra.xlsx : Dates, UK_CPI, CH_CPI, JP_CPI, UK_GDP, CH_GDP, JP_GDP
#
# Sorties :
#   signals_panel.csv   (panel complet)
#   signals_preview.csv (10 dernières obs par paire)
# ------------------------------------------------------------

import pandas as pd
import numpy as np
from pathlib import Path
import warnings, os

warnings.filterwarnings("ignore")

# ---------- Localisation des données ----------
def find_project_root(start=Path.cwd(), markers=("data", ".git", "pyproject.toml", "README.md")):
    start = Path(start).resolve()
    for p in [start, *start.parents]:
        for m in markers:
            if (p / m).exists():
                return p
    return None

env_dir = os.environ.get("FX_DATA_DIR") or os.environ.get("DATA_DIR")
if env_dir:
    DATA_DIR = Path(env_dir).expanduser().resolve()
else:
    root = find_project_root()
    if root:
        DATA_DIR = (root / "data").expanduser().resolve()
    else:
        DATA_DIR = Path(r"H:\Documents\Python\Projet FX\FX_Scoreboard_Project\data").expanduser().resolve()

FX_FILE   = DATA_DIR / "FX.xlsx"
CPI_FILE  = DATA_DIR / "CPI.xlsx"
GDP_FILE  = DATA_DIR / "GDP.xlsx"
CASH_FILE = DATA_DIR / "Cash.xlsx"

_files = {"FX": FX_FILE, "CPI": CPI_FILE, "GDP": GDP_FILE, "CASH": CASH_FILE}
_missing = [n for n, p in _files.items() if not p.exists()]
if _missing:
    raise FileNotFoundError(
        f"Fichiers manquants dans {DATA_DIR}: {', '.join(_missing)}.\n"
        f"Soit définis FX_DATA_DIR/DATA_DIR, soit place les fichiers dans {DATA_DIR}"
    )

print("Using DATA_DIR:", DATA_DIR)

# ---------- Utilitaires ----------
def to_month_end(df, date_col="Dates"):
    df = df.rename(columns={date_col: "Date"}).copy()
    df["Date"] = pd.to_datetime(df["Date"], dayfirst=True, errors="coerce")
    df = df.dropna(subset=["Date"]).set_index("Date").sort_index()
    return df.resample("M").last()

def logret(s, p=1): return np.log(s) - np.log(s.shift(p))
def rolling_z(s, win=60, minp=24):
    mu = s.rolling(win, min_periods=minp).mean().shift(1)
    sd = s.rolling(win, min_periods=minp).std().shift(1)
    return (s - mu) / sd
def winsor(s, p1=0.05, p2=0.95): return s.clip(s.quantile(p1), s.quantile(p2))
def z_std(s): return rolling_z(winsor(s))
def rsi(series, window=14):
    d = series.diff()
    up = d.clip(lower=0).rolling(window).mean()
    dn = (-d.clip(upper=0)).rolling(window).mean()
    rs = up / dn
    return 100 - (100 / (1 + rs))
def bb_z(series, window=20):
    ma = series.rolling(window).mean()
    sd = series.rolling(window).std()
    return (series - ma) / sd

# ---------- 1) Chargement ----------
fx   = to_month_end(pd.read_excel(FX_FILE),  "Dates")
cpi  = to_month_end(pd.read_excel(CPI_FILE), "Dates")
gdp  = to_month_end(pd.read_excel(GDP_FILE), "Dates")
cash = to_month_end(pd.read_excel(CASH_FILE),"Dates")

pairs = [c for c in ["EURUSD","EURGBP","EURCHF","EURJPY"] if c in fx.columns]
fx = fx[pairs].copy()

# ---------- 2) Préparer macro de base (US/EU) ----------
cpi_yoy = cpi[["US_CPI","EU_CPI"]].pct_change(12)
cpi_yoy.columns = ["USD_CPI_yoy","EUR_CPI_yoy"]

gdp_yoy = gdp[["US_GDP","EU_GDP"]].pct_change(12)
gdp_yoy.columns = ["USD_GDP_yoy","EUR_GDP_yoy"]

# ---------- 3) Taux EUR depuis Cash ----------
cash_ret_1m = cash["Cash"].pct_change()
rate_EUR_1m = 12 * cash_ret_1m
rates_df = pd.DataFrame({"EUR_rate_1m": rate_EUR_1m})

# mapping devise → colonne de taux (sera complété par le bypass si Rates.xlsx existe)
ccy_to_ratecol = {"EUR": "EUR_rate_1m", "USD": None, "GBP": None, "CHF": None, "JPY": None}

# ==========================
#      PATCH BYPASS
# ==========================
RATES_FILE = DATA_DIR / "Rates.xlsx"
MACRO_FILE = DATA_DIR / "Macro_extra.xlsx"

# 3.a) Tenter d'ajouter les taux étrangers depuis Rates.xlsx
if RATES_FILE.exists():
    print("[INFO] Rates.xlsx trouvé — chargement des taux étrangers.")
    rates_raw = to_month_end(pd.read_excel(RATES_FILE), "Dates")
    for col in rates_raw.columns:
        if col.lower().endswith("_rate_1m"):
            rates_df[col] = rates_raw[col]
    if "USD_rate_1m" in rates_df.columns: ccy_to_ratecol["USD"] = "USD_rate_1m"
    if "GBP_rate_1m" in rates_df.columns: ccy_to_ratecol["GBP"] = "GBP_rate_1m"
    if "CHF_rate_1m" in rates_df.columns: ccy_to_ratecol["CHF"] = "CHF_rate_1m"
    if "JPY_rate_1m" in rates_df.columns: ccy_to_ratecol["JPY"] = "JPY_rate_1m"
else:
    print("[INFO] Rates.xlsx absent — proxies utilisés pour rate_fore si possible.")

# 3.b) Tenter d'ajouter CPI/GDP UK/CH/JP via Macro_extra.xlsx
cpi_yoy_full = cpi_yoy.copy()
gdp_yoy_full = gdp_yoy.copy()

if MACRO_FILE.exists():
    print("[INFO] Macro_extra.xlsx trouvé — ajout CPI/GDP supplémentaires.")

    # --- CPI sheet ---
    mac_cpi = to_month_end(pd.read_excel(MACRO_FILE, sheet_name="CPI"), "Dates")
    if "UK_CPI" in mac_cpi:
        cpi_yoy_full["GBP_CPI_yoy"] = mac_cpi["UK_CPI"].pct_change(12)
    if "CH_CPI" in mac_cpi:
        cpi_yoy_full["CHF_CPI_yoy"] = mac_cpi["CH_CPI"].pct_change(12)
    if "JP_CPI" in mac_cpi:
        cpi_yoy_full["JPY_CPI_yoy"] = mac_cpi["JP_CPI"].pct_change(12)

    # --- GDP sheet ---
    mac_gdp = to_month_end(pd.read_excel(MACRO_FILE, sheet_name="GDP"), "Dates")
    if "UK_GDP" in mac_gdp:
        gdp_yoy_full["GBP_GDP_yoy"] = mac_gdp["UK_GDP"].pct_change(12)
    if "CH_GDP" in mac_gdp:
        gdp_yoy_full["CHF_GDP_yoy"] = mac_gdp["CH_GDP"].pct_change(12)
    if "JP_GDP" in mac_gdp:
        gdp_yoy_full["JPY_GDP_yoy"] = mac_gdp["JP_GDP"].pct_change(12)

else:
    print("[INFO] Macro_extra.xlsx absent — Macro/Value non-US/EU resteront partielles.")

# ---------- 4) Construire panel long ----------
records = []
for col in pairs:
    base = "EUR"
    fore = col.replace("EUR","")
    s = fx[col].dropna()
    records.append(pd.DataFrame({"pair": col, "base": base, "fore": fore, "spot": s}))
panel = pd.concat(records).sort_index()
panel.index.name = "Date"
panel = panel.reset_index().set_index(["pair","Date"]).sort_index()

# ---------- 5) Merge CPI/GDP (base=EUR, fore selon paire) ----------
def pick(series_df, date_index, colname):
    try:
        return series_df.loc[date_index, colname]
    except Exception:
        return np.nan

def add_macro(panel, cpi_yoy, gdp_yoy):
    out = panel.copy()
    base_cpi, fore_cpi, base_gdp, fore_gdp = [], [], [], []
    for (pair, dt), row in out.iterrows():
        fore = row["fore"]
        # CPI yoy
        base_cpi.append(pick(cpi_yoy, dt, "EUR_CPI_yoy"))
        key_cpi = f"{fore}_CPI_yoy" if f"{fore}_CPI_yoy" in cpi_yoy.columns else None
        fore_cpi.append(pick(cpi_yoy, dt, key_cpi) if key_cpi else np.nan)
        # GDP yoy
        base_gdp.append(pick(gdp_yoy, dt, "EUR_GDP_yoy"))
        key_gdp = f"{fore}_GDP_yoy" if f"{fore}_GDP_yoy" in gdp_yoy.columns else None
        fore_gdp.append(pick(gdp_yoy, dt, key_gdp) if key_gdp else np.nan)
    out["cpi_yoy_base"] = base_cpi
    out["cpi_yoy_fore"] = fore_cpi
    out["gdp_yoy_base"] = base_gdp
    out["gdp_yoy_fore"] = fore_gdp
    return out

# ⚠️ on utilise les versions *full* (avec UK/CH/JP si fournis)
panel = add_macro(panel, cpi_yoy_full, gdp_yoy_full)

# ---------- 6) Taux : vrais si dispo, sinon proxies ----------
# proxy : rate_fore ≈ rate_base + (cpi_yoy_fore - cpi_yoy_base)
def make_rate_fore_proxy(panel_df, rates_df, cpi_yoy_df):
    base_series = rates_df["EUR_rate_1m"]
    prox = []
    for (pair, dt), row in panel_df.iterrows():
        try:
            base_rate = base_series.loc[dt]
        except Exception:
            base_rate = np.nan
        cpi_b = row.get("cpi_yoy_base", np.nan)
        cpi_f = row.get("cpi_yoy_fore", np.nan)
        if pd.notna(base_rate) and pd.notna(cpi_b) and pd.notna(cpi_f):
            prox.append(base_rate + (cpi_f - cpi_b))
        else:
            prox.append(np.nan)
    return pd.Series(prox, index=panel_df.index)

rate_fore_proxy = make_rate_fore_proxy(panel, rates_df, cpi_yoy_full)

rates_df = rates_df.copy()
rates_df.index = pd.to_datetime(rates_df.index)
rates_df = rates_df.sort_index()

def add_rates_with_proxy(panel_df, rates_df, ccy_to_ratecol, rate_fore_proxy):
    out, rb, rf = panel_df.copy(), [], []
    for (pair, dt), row in out.iterrows():
        fore = row["fore"]
        # base
        try: rb_val = rates_df.reindex([dt])["EUR_rate_1m"].values[0]
        except Exception: rb_val = np.nan
        # foreign (réel si présent, sinon proxy)
        colf = ccy_to_ratecol.get(fore)
        if colf and colf in rates_df.columns:
            try: rf_val = rates_df.reindex([dt])[colf].values[0]
            except Exception: rf_val = np.nan
        else:
            try: rf_val = rate_fore_proxy.loc[(pair, dt)]
            except Exception: rf_val = np.nan
        rb.append(rb_val); rf.append(rf_val)
    out["rate_base"], out["rate_fore"] = rb, rf
    return out

panel = add_rates_with_proxy(panel, rates_df, ccy_to_ratecol, rate_fore_proxy)

# ---------- 7) Signaux ----------
panel["ret_1m"]  = panel.groupby(level=0)["spot"].transform(logret)
panel["mom_3m"]  = panel.groupby(level=0)["spot"].transform(lambda s: logret(s, 3))
panel["ma_fast"] = panel.groupby(level=0)["spot"].transform(lambda s: s.rolling(3).mean())
panel["ma_slow"] = panel.groupby(level=0)["spot"].transform(lambda s: s.rolling(12).mean())
panel["ma_signal"] = np.sign(panel["ma_fast"] - panel["ma_slow"])
panel["rsi_14"]    = panel.groupby(level=0)["spot"].transform(lambda s: rsi(s, 14))
panel["rsi_signal"]= (panel["rsi_14"] - 50)/50
panel["z_bb"]      = panel.groupby(level=0)["spot"].transform(lambda s: bb_z(s, 20))

panel["carry_nom_1m"]  = panel["rate_fore"] - panel["rate_base"]
panel["carry_real_1m"] = (panel["rate_fore"] - panel["cpi_yoy_fore"]) - (panel["rate_base"] - panel["cpi_yoy_base"])
panel["policy_bias"]   = (panel.groupby(level=0)["rate_fore"].transform(lambda s: s.rolling(2).mean())
                          - panel.groupby(level=0)["rate_base"].transform(lambda s: s.rolling(2).mean()))
panel["Carry_Score_raw"] = 0.7*panel["carry_real_1m"] + 0.3*panel["carry_nom_1m"]

panel["dSpot_12m"]  = panel.groupby(level=0)["spot"].transform(lambda s: logret(s, 12))
panel["dPPP_12m"]   = panel["cpi_yoy_fore"] - panel["cpi_yoy_base"]
panel["Value_PPP"]  = -(panel["dSpot_12m"] - panel["dPPP_12m"])

panel["gdp_yoy_diff"]   = panel["gdp_yoy_fore"] - panel["gdp_yoy_base"]
panel["cpi_yoy_diff"]   = panel["cpi_yoy_fore"] - panel["cpi_yoy_base"]
panel["Macro_Score_raw"]= 0.5*panel["gdp_yoy_diff"] + 0.5*panel["cpi_yoy_diff"]

panel["hedge_cost_1m"]     = panel["rate_base"] - panel["rate_fore"]
panel["HedgeCost_Score_raw"]= -panel["hedge_cost_1m"]

# ---------- 8) Normalisation ----------
for c_in, c_out in [
    ("Value_PPP","ValueScore"),
    ("Carry_Score_raw","CarryScore"),
    ("mom_3m","Mom_mom"),
    ("ma_signal","Mom_ma"),
    ("rsi_signal","Mom_rsi"),
    ("z_bb","Mom_bb"),
    ("Macro_Score_raw","MacroScore"),
    ("HedgeCost_Score_raw","HedgeCostScore"),
]:
    panel[c_out] = panel.groupby(level=0)[c_in].transform(z_std)

panel["MomentumScore_raw"] = 0.5*panel["Mom_mom"] + 0.2*panel["Mom_ma"] + 0.15*panel["Mom_rsi"] + 0.15*panel["Mom_bb"]
panel["MomentumScore"]     = panel.groupby(level=0)["MomentumScore_raw"].transform(z_std)

# ---------- 9) Agrégation robuste (renormalisation des poids si NaN) ----------
score_cols = ["ValueScore","CarryScore","MomentumScore","MacroScore","HedgeCostScore"]
base_w = pd.Series({"ValueScore":0.25,"CarryScore":0.25,"MomentumScore":0.25,"MacroScore":0.15,"HedgeCostScore":0.10})
sub = panel[score_cols].copy()
mask = sub.notna()
w = pd.DataFrame([base_w]*len(sub), index=sub.index)
w = w * mask
w = w.div(w.sum(axis=1).replace(0, np.nan), axis=0)
panel["FX_Score_raw"] = (sub * w).sum(axis=1)

panel["FX_Score"]      = panel.groupby(level=0)["FX_Score_raw"].transform(lambda s: s.rolling(3, min_periods=1).mean())
panel["FX_Score_next"] = panel.groupby(level=0)["FX_Score"].shift(1)

def hedge_from_score(s, up=0.5, dn=-0.5):
    h = pd.Series(index=s.index, dtype=float)
    h[s >= up] = 0.0
    h[s <= dn] = 1.0
    h[(s > dn) & (s < up)] = 0.5
    return h

panel["HedgeRatio"] = panel.groupby(level=0)["FX_Score_next"].transform(lambda s: hedge_from_score(s))

# ---------- 10) Sorties ----------
keep = [
    "base","fore","spot","ret_1m",
    "rate_base","rate_fore","carry_nom_1m","carry_real_1m","policy_bias","CarryScore",
    "cpi_yoy_base","cpi_yoy_fore","dSpot_12m","dPPP_12m","Value_PPP","ValueScore",
    "mom_3m","ma_fast","ma_slow","ma_signal","rsi_14","rsi_signal","z_bb","MomentumScore",
    "gdp_yoy_base","gdp_yoy_fore","gdp_yoy_diff","cpi_yoy_diff","MacroScore",
    "hedge_cost_1m","HedgeCostScore",
    "FX_Score","FX_Score_next","HedgeRatio"
]
out = panel[keep].copy().reset_index()

out_full = DATA_DIR / "signals_panel.csv"
out_prev = DATA_DIR / "signals_preview.csv"
out.to_csv(out_full, index=False)
out.groupby("pair").tail(10).to_csv(out_prev, index=False)
print(f"[OK] Écrit : {out_full}\n[OK] Écrit : {out_prev}")

# ---------- 11) Diagnostic ----------
diag = (panel[["Value_PPP","Carry_Score_raw","MomentumScore_raw","Macro_Score_raw","HedgeCost_Score_raw"]]
        .notna()
        .groupby(level=0)
        .mean()
        .rename(columns={"Value_PPP":"Value %","Carry_Score_raw":"Carry %","MomentumScore_raw":"Momentum %","Macro_Score_raw":"Macro %","HedgeCost_Score_raw":"HedgeCost %"})
        * 100).round(1)
print("\n=== Couverture des signaux (% non-NaN) ===")
print(diag)


Using DATA_DIR: H:\Documents\Python\Projet FX\FX_Scoreboard_Project\data
[INFO] Rates.xlsx trouvé — chargement des taux étrangers.
[INFO] Macro_extra.xlsx trouvé — ajout CPI/GDP supplémentaires.
[INFO] Rates.xlsx trouvé — chargement des taux étrangers.
[INFO] Macro_extra.xlsx trouvé — ajout CPI/GDP supplémentaires.
[OK] Écrit : H:\Documents\Python\Projet FX\FX_Scoreboard_Project\data\signals_panel.csv
[OK] Écrit : H:\Documents\Python\Projet FX\FX_Scoreboard_Project\data\signals_preview.csv

=== Couverture des signaux (% non-NaN) ===
        Value %  Carry %  Momentum %  Macro %  HedgeCost %
pair                                                      
EURCHF     81.1     67.1        86.6     81.1         85.7
EURGBP     81.4     72.0        86.6     81.4         90.4
EURJPY     81.4     71.1        86.6     80.7         89.4
EURUSD     81.4     77.6        86.6     81.4         96.0
[OK] Écrit : H:\Documents\Python\Projet FX\FX_Scoreboard_Project\data\signals_panel.csv
[OK] Écrit : H:\Doc